In [53]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras.utils import to_categorical

# Preprocessing

In [15]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [16]:
x_train, x_test = x_train/255.0, x_test/255.0

In [17]:
x_train.shape

(50000, 32, 32, 3)

In [51]:
y_train

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

In [54]:
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

In [55]:
y_train_hot

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

# Define the model

In [63]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

# Define custom metric evaluation

In [64]:
def F_beta(y_true , y_pred , beta = 2):
    y_pred = tf.clip_by_value(y_pred , 0 , 1)
    epsilon = tf.keras.backend.epsilon()
    true_positive = tf.reduce_sum(tf.math.round(tf.clip_by_value(y_true * y_pred , 0,1)), axis=1)
    false_positive = tf.reduce_sum(tf.math.round(tf.clip_by_value(y_pred - y_true , 0,1)), axis=1)
    false_negative = tf.reduce_sum(tf.math.round(tf.clip_by_value(y_true - y_pred , 0,1)), axis=1)
    
    P = true_positive / (true_positive + false_positive + epsilon)
    R = true_positive / (true_positive + false_negative + epsilon)
    
    return tf.reduce_mean(((1+beta**2)*P*R) / beta**2 * P + R + epsilon)

In [65]:
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01),
            loss='categorical_crossentropy',
            metrics=[F_beta])

In [66]:
history = model.fit(x_train , y_train_hot ,  epochs=50 , validation_split=0.2)

Epoch 1/50
1250/1250 [==============================] - 4s 2ms/step - loss: 1.9910 - F_beta: 0.0528 - val_loss: 1.8452 - val_F_beta: 0.1233
Epoch 2/50
1250/1250 [==============================] - 3s 2ms/step - loss: 1.8015 - F_beta: 0.1805 - val_loss: 1.7548 - val_F_beta: 0.2408
Epoch 3/50
1250/1250 [==============================] - 3s 2ms/step - loss: 1.7201 - F_beta: 0.2719 - val_loss: 1.6972 - val_F_beta: 0.2965
Epoch 4/50
1250/1250 [==============================] - 3s 2ms/step - loss: 1.6697 - F_beta: 0.3382 - val_loss: 1.7005 - val_F_beta: 0.3149
Epoch 5/50
1250/1250 [==============================] - 3s 2ms/step - loss: 1.6308 - F_beta: 0.3804 - val_loss: 1.6763 - val_F_beta: 0.3520
Epoch 6/50
1250/1250 [==============================] - 3s 2ms/step - loss: 1.5965 - F_beta: 0.4217 - val_loss: 1.6584 - val_F_beta: 0.3938
Epoch 7/50
1250/1250 [==============================] - 3s 2ms/step - loss: 1.5667 - F_beta: 0.4618 - val_loss: 1.6302 - val_F_beta: 0.3756
Epoch 8/50
1250/1250